In [3]:
Docstring:     
Class to perform random over-sampling.

Object to over-sample the minority class(es) by picking samples at random
with replacement.

무작위 오버샘플링(Random Over-Sampling) 을 수행하기 위한 클래스
소수 클래스(들)의 샘플을 복원 추출(with replacement)을 통해 무작위로 선택하여 데이터를 증강
        
    
Parameters
----------
sampling_strategy : float, str, dict or callable, (default='auto')
    Sampling information to resample the data set.

    - When ``float``, it corresponds to the desired ratio of the number of
      samples in the minority class over the number of samples in the
      majority class after resampling. Therefore, the ratio is expressed as
      :math:`\alpha_{os} = N_{rm} / N_{M}` where :math:`N_{rm}` is the
      number of samples in the minority class after resampling and
      :math:`N_{M}` is the number of samples in the majority class.

        .. warning::
           ``float`` is only available for **binary** classification. An
           error is raised for multi-class classification.

    - When ``str``, specify the class targeted by the resampling. The
      number of samples in the different classes will be equalized.
      Possible choices are:

        ``'minority'``: resample only the minority class;

        ``'not minority'``: resample all classes but the minority class;

        ``'not majority'``: resample all classes but the majority class;

        ``'all'``: resample all classes;

        ``'auto'``: equivalent to ``'not majority'``.

    - When ``dict``, the keys correspond to the targeted classes. The
      values correspond to the desired number of samples for each targeted
      class.

    - When callable, function taking ``y`` and returns a ``dict``. The keys
      correspond to the targeted classes. The values correspond to the
      desired number of samples for each class.

    
샘플링할 데이터 세트에 대한 정보를 설정

float인 경우
소수 클래스의 샘플 수를 다수 클래스의 샘플 수에 대한 비율로 설정
⚠️ 단, 이 방식은 이진 분류(binary classification) 에서만 사용 가능, 다중 클래스에서는 오류 발생

str인 경우
리샘플링할 클래스를 문자열로 지정합니다. 선택 가능한 값:
'minority': 소수 클래스만 리샘플링
'not minority': 소수 클래스를 제외한 모든 클래스 리샘플링
'not majority': 다수 클래스를 제외한 모든 클래스 리샘플링
'all': 모든 클래스 리샘플링
'auto': 'not majority'와 동일

dict인 경우
키는 리샘플링 대상 클래스이며, 값은 해당 클래스의 목표 샘플 수

callable인 경우
y를 받아서 dict를 반환하는 함수입니다. 키와 값은 위의 dict와 동일한 의미를 가짐.
        


Attributes
----------
sample_indices_ : ndarray, shape (n_new_samples)
    Indices of the samples selected.

    .. versionadded:: 0.4
       ``sample_indices_`` used instead of ``return_indices=True``.
    
리샘플링된 샘플들의 인덱스
✔️ 0.4 버전부터 추가되었으며, return_indices=True 대신 사용합니다.

Notes
-----
Supports multi-class resampling by sampling each class independently.
Supports heterogeneous data as object array containing string and numeric data.

클래스마다 독립적으로 샘플링하여 다중 클래스 리샘플링을 지원
문자열과 숫자가 섞인 데이터(heterogeneous data) 도 처리 가능     

In [1]:
import pandas as pd
import numpy as np

np.random.seed(1234)

n = 100

data = {
    'age': np.random.randint(18, 65, size=n),  
    'salary': np.random.normal(loc=50000, scale=15000, size=n).astype(int),  
    'department': np.random.choice(['HR', 'Engineering', 'Marketing'], size=n, p=[0.7, 0.2, 0.1])
}

# 데이터프레임 생성
df = pd.DataFrame(data)

df['department'].value_counts()

HR             72
Engineering    19
Marketing       9
Name: department, dtype: int64

In [2]:
from imblearn.over_sampling import RandomOverSampler

# auto (default)
# minority = 소수 클래스만 다수 클래스 수준까지 복제 
# 'not minority' = 소수 클래스 이외의 클래스를 모두 복제
# 'not majority' = 다수 클래스 이외의 클래스를 모두 복제
# 'all' = 모든 클래스를 가장 많은 클래스 수준까지 복제
# dict = 원하는 클래스별로 구체적인 수를 지정
# float = 전체 데이터의 소수 클래스 비율을 정수 비율로 맞춤 (ex. 0.5 → minority / majority = 0.5)

oversample = RandomOverSampler(sampling_strategy='minority', random_state=1234)
X_train, y_train = oversample.fit_resample(df[['age', 'salary']], df['department']) # fit_resample(data, class)

In [36]:
df_resampled = pd.DataFrame(X_train, columns=['age', 'salary'])
df_resampled['department'] = y_train

In [37]:
df_resampled['department'].value_counts()

Marketing      72
HR             72
Engineering    19
Name: department, dtype: int64

In [10]:
# 모든 소수 클래스가 최대 다수 클래스가 되도록 설정
target_counts = {
    'Engineering': df['department'].value_counts()['HR'],  
    'Marketing': df['department'].value_counts()['HR']
}

oversample = RandomOverSampler(sampling_strategy=target_counts, random_state=1234)
X_train, y_train = oversample.fit_resample(df[['age', 'salary']], df['department'])

In [11]:
df_resampled = pd.DataFrame(X_train, columns=['age', 'salary'])
df_resampled['department'] = y_train

In [12]:
df_resampled['department'].value_counts()

HR             72
Engineering    72
Marketing      72
Name: department, dtype: int64